# 这是一个汇总文档

## 文档的创建和实例

In [1]:
from mongoengine import connect, disconnect
from mongoengine import Document
from mongoengine import StringField, IntField
from mongo_config import TEST_DB1, TEST_DB2, TEST_DB3, HOST, PORT, USERNAME, PASSWORD
from mongoengine import BooleanField, DateTimeField, DictField, EmailField, FloatField, ListField, ReferenceField
from mongoengine import EmbeddedDocument, EmbeddedDocumentField, LazyReferenceField
from datetime import datetime

In [2]:
# 连接到已创建的数据库
connect(TEST_DB1, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB1, alias=TEST_DB1)
connect(TEST_DB2, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB2, alias=TEST_DB2)
connect(TEST_DB3, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB3, alias=TEST_DB3)

MongoClient(host=['192.168.2.172:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), authsource='engine_learn3', uuidrepresentation=3)

### 查询要使用的全字段文档

In [3]:
class BookType(Document):
    type_name = StringField()
    meta = {"db_alias": TEST_DB1}

In [4]:
class BookAuthor(Document):
    name = StringField(unique=True)
    age = StringField()
    meta = {"db_alias": TEST_DB1}

In [5]:
class Seller(Document):
    name = StringField(unique=True)
    phone = StringField(regex="^\d+")
    meta = {"db_alias": TEST_DB1}

In [6]:
class PublishingHouse(EmbeddedDocument):
    name =  StringField(unique=True)
    address =  StringField()

In [7]:
class BookAllField(Document):
    bookid = StringField(unique=True)
    name = StringField()
    ifnewbook = BooleanField()  # 是否为新上架图书
    shelves_date = DateTimeField()
    author = ReferenceField(BookAuthor)  # 使引用字段执行其他集合
    author_email = EmailField()
    price = FloatField()
    book_type = ReferenceField(BookType, dbref=True)  # 将引用ref文档
    publishing_house = EmbeddedDocumentField(PublishingHouse)
    seller = LazyReferenceField(Seller, dbref=True)
    tag = ListField()
    meta = {"db_alias": TEST_DB1}

In [16]:
from typing import List
def create_book(
        bookid: str, book_name: str, ifnewbook: bool, shelves_date: datetime, price: float, 
        tag: List[str], author_email: str, author_name: str,  author_age: str, 
        seller_name: str, seller_phone: str, book_type: str, publish_name: str, publish_address: str
    ):
    book = BookAllField(bookid=bookid, name = book_name, ifnewbook = ifnewbook, 
            shelves_date = shelves_date, price = price, tag = tag, author_email = author_email,
            publishing_house = PublishingHouse(name=publish_name, address=publish_address)
        )
    
    the_author = BookAuthor.objects(name=author_name).first()
    if the_author is None:
        the_author = BookAuthor(name=author_name, age=author_age).save()
    book.author = the_author
    
    the_seller = Seller.objects(name=seller_name).first()
    if the_seller is None:
        the_seller = Seller(name=seller_name, phone=seller_phone).save()
    book.seller = the_seller
    
    
    the_book_type = BookType.objects(type_name=book_type).first()
    if the_book_type is None:
        the_book_type = BookType(type_name=book_type).save()
    book.book_type = the_book_type
     
    book.save()

In [9]:
# 模拟数据
import random
from faker import Faker
fkdata = Faker(["ZH_cn"])

In [31]:
# 模拟数据
fkdata.name()
fkdata.address()
fkdata.phone_number()
fkdata.company()
fkdata.email()
fkdata.date()
fkdata.boolean()

'谢秀梅'

In [11]:
def random_create():
    book_name = "".join(fkdata.words(random.randint(1, 3)))
    create_book(
            bookid=fkdata.ssn(), book_name=book_name, ifnewbook=fkdata.boolean(), shelves_date=fkdata.date(), price=random.randint(100, 20000)/100, 
            tag=fkdata.words(random.randint(1, 3)), author_email=fkdata.email(), author_name=fkdata.name(),  author_age=random.choice(dynasty.split("、")), 
            seller_name=fkdata.company(), seller_phone=fkdata.phone_number(), book_type=fkdata.word(), publish_name=fkdata.company(), publish_address=fkdata.address()
        )

In [10]:
dynasty = "夏朝、商朝、西周、东周、秦朝、西楚、西汉、新朝、玄汉、东汉、三国、曹魏、蜀汉、孙吴、西晋、东晋、十六国、南朝、刘宋、南齐、南梁、南陈、北朝、北魏、东魏、北齐、西魏、北周、隋朝、唐朝、武周、唐朝中后期、五代、后梁、后唐、后晋、后汉、后周、十国、南吴、南唐、前属、后属、南汉、南楚、吴越、闽国、荆国、北汉、北宋、南宋、辽朝、西夏、金朝、元朝、明朝、清朝"

In [20]:
from tqdm import tqdm
for _ in tqdm(range(10000)):
    random_create()

100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [05:44<00:00, 29.03it/s]


In [34]:
import sys
from importlib import reload
stdi, stdo, stde = sys.stdin, sys.stdout, sys.stderr
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdin, sys.stdout, sys.stderr = stdi, stdo, stde

AttributeError: module 'sys' has no attribute 'setdefaultencoding'

### 基本查询-过滤查询

The query may be filtered by calling the `QuerySet` object with field lookup keyword arguments. The keys in the keyword arguments correspond to fields on the Document you are querying:

In [27]:
books = BookAllField.objects(name="支持这种")

In [31]:
for book in books:
    print(book.to_json())

{"_id": {"$oid": "64087d72cbd07ad5619989b5"}, "bookid": "511600193907182680", "name": "\u652f\u6301\u8fd9\u79cd", "ifnewbook": true, "shelves_date": {"$date": 42854400000}, "author": {"$oid": "64087d72cbd07ad5619989b2"}, "author_email": "luojuan@example.org", "price": 55.46, "book_type": {"$ref": "book_type", "$id": {"$oid": "64087d72cbd07ad5619989b4"}}, "publishing_house": {"name": "\u6234\u7855\u7535\u5b50\u79d1\u6280\u6709\u9650\u516c\u53f8", "address": "\u9655\u897f\u7701\u6dd1\u5170\u5e02\u5408\u5ddd\u6f6e\u5dde\u8defT\u5ea7 131033"}, "seller": {"$ref": "seller", "$id": {"$oid": "64087d72cbd07ad5619989b3"}}, "tag": ["\u5305\u62ec", "\u5176\u4ed6"]}


In [29]:
book.to_json?

Signature: book.to_json(*args, **kwargs)
Docstring:
Convert this document to JSON.

:param use_db_field: Serialize field names as they appear in
    MongoDB (as opposed to attribute names on this document).
    Defaults to True.
File:      c:\users\kent\appdata\local\packages\pythonsoftwarefoundation.python.3.8_qbz5n2kfra8p0\localcache\local-packages\python38\site-packages\mongoengine\base\document.py
Type:      method